In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle datasets download -d paultimothymooney/blood-cells # api copied from kaggle

 82% 89.0M/108M [00:01<00:00, 66.9MB/s]
100% 108M/108M [00:01<00:00, 93.0MB/s] 


In [ ]:
!unzip "blood-cells.zip" -d "/content"

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report

In [4]:
train_dir = 'dataset2-master/dataset2-master/images/TRAIN'
test_dir = 'dataset2-master/dataset2-master/images/TEST'

In [5]:
# Create generators

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input , 
    validation_split= 0.2
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input , 
    validation_split= 0.2
)

In [6]:
# Flow image data 

train_images = train_gen.flow_from_directory(
    directory = train_dir , target_size = (224,224) , color_mode = 'rgb' , 
    class_mode = 'categorical' , batch_size = 32 , shuffle= True , seed = 42,
    subset = 'training'
)

val_images = train_gen.flow_from_directory(
    directory = train_dir , target_size = (224,224) , color_mode = 'rgb' , 
    class_mode = 'categorical' , batch_size = 32 , shuffle= True , seed = 42,
    subset = 'validation'
)

test_images = test_gen.flow_from_directory(
    directory = test_dir , target_size = (224,224) , color_mode = 'rgb' , 
    class_mode = 'categorical' , batch_size = 32 , shuffle= False , seed = 42
)

Found 7968 images belonging to 4 classes.
Found 1989 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.


In [7]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [8]:
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128,activation = 'relu')(pretrained_model.output)

outputs = tf.keras.layers.Dense(4,activation='sigmoid')(x)

model = tf.keras.Model(inputs = inputs , outputs = outputs)

model.compile(
    optimizer = 'adam' , 
    loss = 'categorical_crossentropy' , 
    metrics = ['accuracy']
)

In [9]:
# Training 

history = model.fit(
    train_images , 
    validation_data = val_images ,
    epochs = 25 , 
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_loss' , 
            patience = 3,
            restore_best_weights = True
        )
    ]
    
)

Epoch 1/25
249/249 [==============================] - 59s 98ms/step - loss: 1.0832 - accuracy: 0.5561 - val_loss: 0.5817 - val_accuracy: 0.7753
Epoch 2/25
249/249 [==============================] - 23s 93ms/step - loss: 0.5130 - accuracy: 0.8112 - val_loss: 0.4705 - val_accuracy: 0.8195
Epoch 3/25
249/249 [==============================] - 23s 92ms/step - loss: 0.3543 - accuracy: 0.8707 - val_loss: 0.3671 - val_accuracy: 0.8607
Epoch 4/25
249/249 [==============================] - 23s 92ms/step - loss: 0.2982 - accuracy: 0.8847 - val_loss: 0.4382 - val_accuracy: 0.8190
Epoch 5/25
249/249 [==============================] - 23s 91ms/step - loss: 0.2328 - accuracy: 0.9147 - val_loss: 0.2631 - val_accuracy: 0.9060
Epoch 6/25
249/249 [==============================] - 23s 91ms/step - loss: 0.1779 - accuracy: 0.9370 - val_loss: 0.3332 - val_accuracy: 0.8658
Epoch 7/25
249/249 [==============================] - 23s 91ms/step - loss: 0.1493 - accuracy: 0.9502 - val_loss: 0.2376 - val_accuracy:

In [11]:
from keras.preprocessing import image

img_width, img_height = 224, 224
img = image.load_img('/content/dataset2-master/dataset2-master/images/TEST_SIMPLE/LYMPHOCYTE/_1_4044.jpeg', target_size = (img_width, img_height, 3))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
np.argmax(model.predict(img), axis=1)

array([1])

In [13]:
model.save("model1.h5")